In [1]:
import os,sys,tqdm
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *

from collections import Counter
import pandas as pd
import shutil
import pickle
import gc
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

In [2]:
max_words = 8352#8352 #Top most frequent words to consider. Any less frequent word will appear as oov_char value in the sequence data.
max_length = 360#360

In [3]:
word_index = reuters.get_word_index()
print('all_words#:',len(word_index))
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,maxlen=max_length,
                                                         test_split=0.2,seed=830913)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
word_index

all_words#: 30979
8260 train sequences
2066 test sequences


{'mdbl': 10996,
 'fawc': 16260,
 'degussa': 12089,
 'woods': 8803,
 'hanging': 13796,
 'localized': 20672,
 'sation': 20673,
 'chanthaburi': 20675,
 'refunding': 10997,
 'hermann': 8804,
 'passsengers': 20676,
 'stipulate': 20677,
 'heublein': 8352,
 'screaming': 20713,
 'tcby': 16261,
 'four': 185,
 'grains': 1642,
 'broiler': 20680,
 'wooden': 12090,
 'wednesday': 1220,
 'highveld': 13797,
 'duffour': 7593,
 '0053': 20681,
 'elections': 3914,
 '270': 2563,
 '271': 3551,
 '272': 5113,
 '273': 3552,
 '274': 3400,
 'rudman': 7975,
 '276': 3401,
 '277': 3478,
 '278': 3632,
 '279': 4309,
 'dormancy': 9381,
 'errors': 7247,
 'deferred': 3086,
 'sptnd': 20683,
 'cooking': 8805,
 'stratabit': 20684,
 'designing': 16262,
 'metalurgicos': 20685,
 'databank': 13798,
 '300er': 20686,
 'shocks': 20687,
 'nawg': 7972,
 'tnta': 20688,
 'perforations': 20689,
 'affiliates': 2891,
 '27p': 20690,
 'ching': 16263,
 'china': 595,
 'wagyu': 16264,
 'affiliated': 3189,
 'chino': 16265,
 'chinh': 16266,
 '

In [13]:
# train_len = [len(x) for x in x_train]
# test_len = [len(x) for x in x_test]
# all_len = train_len
# all_len.extend(test_len)
# basic_statistics(all_len)

,length
mean,145.964197
std,145.878476
mode,17.000000
min,2.000000
q1,60.000000
median,95.000000
q3,180.000000
max,2376.000000
iqr,120.000000
outlier,360.000000


In [15]:
# df = pd.DataFrame(all_len)
# df.to_excel('./results/length_dist.xlsx', header=False, index=False)
# df

In [21]:
# train_words = []
# for x in x_train:
#     train_words.extend(x)
# test_words = []
# for x in x_test:
#     test_words.extend(x)
# all_words = train_words
# all_words.extend(test_words)
# all_statistcs = Counter(all_words)
# all_statistcs

Counter({1: 11228,
         53: 4213,
         352: 647,
         26: 8451,
         14: 15015,
         279: 801,
         39: 5818,
         72: 3091,
         4497: 26,
         18: 11039,
         83: 2597,
         5291: 21,
         88: 2381,
         5397: 20,
         11: 20141,
         3412: 37,
         19: 10755,
         151: 1363,
         230: 962,
         831: 253,
         15: 13329,
         165: 1232,
         318: 707,
         3780: 33,
         124: 1676,
         1527: 117,
         1424: 128,
         35: 6588,
         5302: 20,
         12: 16668,
         17: 11191,
         486: 459,
         341: 663,
         142: 1466,
         255: 870,
         219: 997,
         429: 528,
         68: 3363,
         146: 1402,
         252: 882,
         191: 1098,
         15448: 3,
         3631: 35,
         2283: 65,
         71: 3120,
         10: 29581,
         342: 660,
         49: 4565,
         1977: 80,
         324: 695,
         27: 8311,
         9222: 

In [28]:
# df = pd.DataFrame.from_dict(dict(all_statistcs), orient = 'index')
# df.to_excel('./results/words_dist2.xlsx', header=False, index=True)
# df

,0
1,11228
53,4213
352,647
26,8451
14,15015
...,...
24452,1
18567,2
27222,1
26864,1


In [4]:
trainX = tf.keras.preprocessing.sequence.pad_sequences(x_train,maxlen=max_length,padding='post',value=0)
testX = tf.keras.preprocessing.sequence.pad_sequences(x_test,maxlen=max_length,padding='post',value=0)
print(trainX.shape, testX.shape)

(8260, 360) (2066, 360)


In [5]:
hidden_dim = 128
# do = 0.1

## Graph execution
### Embedder

In [42]:
int_id = Input(shape=(max_length,), dtype='int32', name='int_ids') # 輸入的api funvtion name ID
int_ids = Masking(mask_value=0)(int_id)
sent_emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')(int_ids) 

### Encoder

In [43]:
rnn = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)(sent_emb)
rnn = BatchNormalization(name='bn')(rnn)


### Filter

In [44]:
fil = TimeDistributed(Dense(1,activation='sigmoid',
                             name='filter_out'),name='TD2')(rnn)

### Classfier

In [47]:
mul = Multiply()([fil,sent_emb])
clf = LSTM(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm')(mul)
clf = BatchNormalization(name='bn3')(clf)
clf = Dense(max(y_train)+1,activation='softmax',name='clf')(clf)

## Compile

In [48]:
model = Model(inputs=int_id, outputs = clf)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
int_ids (InputLayer)            [(None, 358)]        0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 358)          0           int_ids[0][0]                    
__________________________________________________________________________________________________
glove_emb (Embedding)           (None, 358, 128)     1280000     masking_1[0][0]                  
__________________________________________________________________________________________________
common_extract (GRU)            (None, 358, 64)      37248       glove_emb[0][0]                  
______________________________________________________________________________________________

In [53]:
# loss
import keras.backend as K
def custom_objective(layer):
    return K.sum(layer.output)
#     return K.sum(layer.output)
# kk = tf.keras.backend.ea
model.compile(loss=custom_objective(model.get_layer(name='TD2')),optimizer='adam')

Using TensorFlow backend.


OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

## Eager Execution

In [52]:
'''# whole model
do = 0
init = tensorflow.keras.initializers.Ones()
class base_model(Model):
    def __init__(self):
        super(base_model, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.bn1 = BatchNormalization(name='bn1')
        self.fil = Dense(1,activation='hard_sigmoid',kernel_initializer=init,bias_initializer=init,name='filter_out')
        #self.fil = TimeDistributed(Dense(1,activation='sigmoid', name='filter_out'),name='TD2')
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = LSTM(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')
    def transform(self,x):
        return tf.math.round(x)
    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.rnn1(x1)
        x = self.bn1(x)
        y = self.fil(x)
        y1 = self.transform(y)
        x2 = self.mul([y1,x1])
        x = self.rnn2(x2) #x
        x = self.bn2(x)
        y2 = self.out(x)
        return y,y1,y2
        #return y,y1,y2,x2
        
model = base_model()
'''

In [277]:
# partial1 model
# init_w = tensorflow.keras.initializers.Constant(value=0.9) #portyion=0.6, w=0.9, b = 0.8-0.85 (0.83從0開始)
init_w = tensorflow.keras.initializers.RandomNormal(mean=1.667,stddev=0.2556)
# init_b = tensorflow.keras.initializers.Constant(value=0.84) #w=1 ; b=0.499, portion=1
init_b = tensorflow.keras.initializers.RandomNormal(mean=1.667,stddev=0.2756)
def onezero(x):
    portion = 0.6#0.6#0.6 #0.6~1
    z = tf.where(x>=1.0, x - x + 1.0, x)
    y = tf.where(z<=0.0, z - z + 0.0, portion*z)
    return y

class base_model_1(Model):
    def __init__(self):
        super(base_model_1, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/4),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.att = Attention(name='selfatt')
        self.bn1 = BatchNormalization(name='bn1')
        #self.fil = Dense(1,activation=onezero,name='filter_out')
        self.fil = TimeDistributed(Dense(1,activation=onezero,kernel_initializer=init_w,bias_initializer=init_b, name='filter_out'),name='TD2') #relu/linear/step function

    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.att([x1,x1])
        #x = self.rnn1(x1)
        #x = self.bn1(x)
        y = self.fil(x)
        return x1,y

model1 = base_model_1()
#phase2
# model3 = load_model(saveP)

# model3 = base_model_1()
# model3.load_weights('./model/2019110501/model1')#,by_name=True)
# model1.emb.set_weights(model3.emb.get_weights())

In [278]:
# partial2 model
class base_model_2(Model):
    def __init__(self):
        super(base_model_2, self).__init__()
        self.mul = Multiply()
        #self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = GRU(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')

    def call(self,x1,y1):
        x2 = self.mul([y1,x1])
        x = self.rnn3(x2) #x2 #y1=weight|binary
        x = self.bn2(x)
        y2 = self.out(x)
        return y2
    
model2 = base_model_2()

In [448]:
'''
# TEST
x = tf.random.uniform((32, 6))
out1,out2 = model1(x)
out2 = tf.squeeze(out2,axis=-1)
# out2 = out2.astype('float32')
# out2 = tf.dtypes.cast(out2, tf.float8)
# print(out2.shape)
yy = tf.matmul(out2,kk)
print(yy[0].shape)
tf.where(yy[0]==0,1,0)

gg = tf.range(-5,5)
gg = tf.expand_dims(gg,axis=0)
gg = tf.keras.backend.repeat_elements(gg,rep=32,axis=0)
tf.where(gg==1,gg,0)

kk = tf.Variable(np.array([[1.0,1.0,1.0,0.0,0.0,0.0],[0.0,1.0,1.0,1.0,0.0,0.0],[0.0,0.0,1.0,1.0,1.0,0.0],
                           [0.0,0.0,0.0,1.0,1.0,1.0]]).T,dtype='float32')
kk = tf.expand_dims(kk,axis=0)
kk = tf.keras.backend.repeat_elements(kk,rep=1,axis=0)
kk.shape
'''

(32, 4)


<tf.Tensor: id=5770753, shape=(32, 4), dtype=int32, numpy=
array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]], dtype=int32)>

In [246]:
# 記得要跑到一個好的data
# batch_size = 128 #,reshuffle_each_iteration=True
# train_ds = tf.data.Dataset.from_tensor_slices((trainX,y_train)).shuffle(trainX.shape[0],reshuffle_each_iteration=True).batch(batch_size)
# valid_ds = tf.data.Dataset.from_tensor_slices((testX,y_test)).batch(batch_size)

In [279]:
seq_num = 2 #連續幾個才叫做連續，要改modify_idx看有幾個

arr_len = max_length - seq_num + 1
seq_arr = []
for i in range(arr_len):
    ori_np = np.array([0]*max_length)
    modify_idx = [i,i+1] #要跟著seq_num改
    ori_np[modify_idx]=1
    seq_arr.append(ori_np)
seq_arr = np.array(seq_arr)
seq_mask = tf.Variable(seq_arr.T,dtype='float32')
seq_mask = tf.expand_dims(seq_mask,axis=0)
seq_mask

<tf.Tensor: id=23103458, shape=(1, 360, 359), dtype=float32, numpy=
array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)>

In [280]:
def loss_object1(predictions): #filter loss
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def one_percentage(predictions): #1 num
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def seq_loss(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    predictions = tf.squeeze(predictions,axis=-1)
    results = tf.matmul(predictions,seq_mask)
    results = tf.where(results==seq_num,1.0,0.0)
    loss_ = tf.reduce_mean(results)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def word_reproduce1(prediction_int):
    return prediction_int
def word_reproduce2(prediction_int):
    return prediction_int
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy() #clf loss

optimizer1 = tf.keras.optimizers.SGD(learning_rate=0.004, momentum=0.9, nesterov=True)
# optimizer1 = tf.keras.optimizers.Nadam()#Adam(learning_rate=0.0003)
# optimizer2 = tf.keras.optimizers.RMSprop()

train_loss = tf.keras.metrics.Mean(name='train_loss') #total_loss
train_accloss = tf.keras.metrics.Mean(name='train_accloss')#loss_acc
train_filloss = tf.keras.metrics.Mean(name='train_filloss') #loss_filter
train_seqloss = tf.keras.metrics.Mean(name='train_seqloss') #loss_seq
train_ones = tf.keras.metrics.Mean(name='train_ones') #ones_num
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') #acc_rate

test_loss = tf.keras.metrics.Mean(name='test_loss') #total_loss
test_accloss = tf.keras.metrics.Mean(name='test_accloss')#loss_acc
test_filloss = tf.keras.metrics.Mean(name='test_filloss') #loss_filter
test_seqloss = tf.keras.metrics.Mean(name='test_seqloss')
test_ones = tf.keras.metrics.Mean(name='test_ones') #ones_num
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [281]:
# seperate partial model
alpha = 0.000005#0.000002 #pahse1: -0.1 / 0.0 ; phase2: 0.01~0.05~0.1 有1-就是希望1越多，沒1-就是希望0越多1越少
beta = 1.0 #clf loss 越大越要求分好
gamma = 0.0 #seqloss 越大越要求連續
#-0.001 / 1.0 / 1.0

@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=False) as tape:
        emb, pred_imp = model1(x)
        #loss1 = alpha*loss_object1(pred_imp) #phase1
        #pred_imp2 = tf.math.round(pred_imp)
        #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
        pred_imp2 = tf.math.round(pred_imp)
        loss1 = loss_object1(pred_imp) #有1-就是希望1越多，沒1-就是希望0越多1越少 #pahse2:alpha*loss_object1(pred_imp) ; phase1: alpha*(1-loss_object1(pred_imp))
        pred_cat = model2(emb,pred_imp2) #weight: pred_imp; 0/1binary: pred_imp2 。#pahse1: pred_imp; phase2; pred_imp2
        loss2 = loss_object2(yc, pred_cat)
        loss3 = 1-seq_loss(pred_imp2)
        loss = alpha*loss1 + beta*loss2 + gamma*loss3
    trainable_variable = model1.trainable_variables
    trainable_variable.extend(model2.trainable_variables)
    gradients = tape.gradient(loss,trainable_variable)
    optimizer1.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss) #total_loss
    train_filloss(loss1)
    train_accloss(loss2)
    train_seqloss(loss3) #loss_seq
    train_accuracy(yc, pred_cat) #acc_rate
    ones = one_percentage(pred_imp2) #pred_imp2
    train_ones(ones) #ones_num
    
    
@tf.function
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #weight: pred_imp ; binary:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)

    
word_idx_pd = pd.DataFrame(data=word_index,index=["ID"]).T
word_idx_pd['ori_word'] = word_idx_pd.index 
word_idx_pd = word_idx_pd.set_index(["ID"])
print(word_idx_pd)


# def getKeysByValues(listOfValues,dfOfElements=word_idx_pd):
#     ori_word_li = []
#     for item in listOfValues:
#         key = item.numpy()
#         if key == 0:
#             continue
#         ori_word = dfOfElements.loc[key][0]
#         ori_word_li.append(ori_word)
#     return ori_word_li
ep = -1
@tf.function
def test_step_inf(x,yc,epoch):
    #ori_text = getKeysByValues(list(x))
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    #if ep != epoch:
        #ep = epoch
        #x = tf.dtypes.cast(x,dtype=tf.float32)
    pred_imp3 = tf.squeeze(pred_imp2)
    pred_imp3 = tf.dtypes.cast(pred_imp3,dtype=tf.int32)
    used_x = tf.math.multiply(x,pred_imp3)
        #word_reproduce1(x)
        #word_reproduce2(used_x)
        #used_text = getKeysByValues(list(used_x))
        #tf.print(used_x)
        #print(emb.numpy(),pred_imp, x,pred_imp3,used_x)
        #int_ori_text.append(x)
        #int_fil_text.append(used_x)
    
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #phase1: pred_imp ; phase2:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)
    return x, used_x

        ori_word
ID              
10996       mdbl
16260       fawc
12089    degussa
8803       woods
13796    hanging
...          ...
16258    rotting
10995       pods
2849       emery
30979  northerly
16259   onomichi

[30979 rows x 1 columns]


In [331]:
'''
#AIO
alpha = 0.1
beta = 1
gamma = 0
@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=True) as tape: #persistent=True
        pred_imp,pred_round , pred_cat = model(x)
#         pred_cat = model(x)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
        loss1 = alpha*loss_object1(pred_imp)
        loss2 = beta*loss_object2(yc,pred_cat)
        loss = loss_object2(yc,pred_cat)
#     gradients = tape.gradient(loss, model.trainable_variables)
    grad1 = tape.gradient(loss1, model.trainable_variables)
    grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer1.apply_gradients(zip(gradients, model.trainable_variables))
    optimizer1.apply_gradients(zip(grad1, model.trainable_variables))
    optimizer2.apply_gradients(zip(grad2, model.trainable_variables))
#     with tf.GradientTape() as tape:
#         pred_imp , pred_cat = model(x)
#         loss2 = loss_object2(yc,pred_cat)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
#     grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer2.apply_gradients(zip(grad2, model.trainable_variables))

    train_loss(loss)
    train_accuracy(yc, pred_cat)
    ones = one_percentage(pred_round)
    train_ones(ones)
    
@tf.function
def test_step(x,yc):
    pred_imp,pred_round, pred_cat = model(x)
#     pred_cat = model(x)
    t_loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat) 
#     t_loss = loss_object2(yc,pred_cat)
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_round)
    test_ones(t_ones)
'''    

In [ ]:
EPOCHS = 2000
DateID = '2019112702'

saveP1 = './model/'+DateID+'/model1'
saveP2 = './model/'+DateID+'/model2'
train_loss_acc = []
train_loss_filter = []
train_loss_seq = []
train_weighted_loss = []
train_acc_rate = []
train_ones_num = []

test_loss_acc = []
test_loss_filter = []
test_loss_seq = []
test_weighted_loss = []
test_acc_rate = []
test_ones_num = []

int_ori_text =[]
int_fil_text = []
int_label = []
# signature_dict = {'att':model1.att}

gc.collect()
best_clf = 0.0
for epoch in range(EPOCHS):
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        ori,fil = test_step_inf(test_text, test_labels,epoch) #with restore words
        #print(k2)
        #test_step(test_text, test_labels) #original
    int_ori_text.append(ori.numpy())
    int_fil_text.append(fil.numpy())
    int_label.append(test_labels.numpy())
    
    template = 'Epoch {}, Total Loss: {}, Clf Loss: {}, Filter Loss: {}, Seq Loss: {}, Accuracy Rate: {:5.2f}%, Ones Portion: {}, \
            Test_Total_Loss: {}, Test_Clf_Loss: {}, Test_Filter_Loss: {}, TEST_Seq_Loss: {}, Test_Accuracy_Rate: {:5.2f}%, Test_Ones_Portion: {}'
    print(template.format(epoch+1,train_loss.result(),
                          train_accloss.result(),train_filloss.result(),train_seqloss.result(),
                        train_accuracy.result()*100,train_ones.result(),
                        test_loss.result(),
                        test_accloss.result(),test_filloss.result(),test_seqloss.result(),
                        test_accuracy.result()*100,test_ones.result(),
                        ))
    #int_ori_text.append(word_reproduce1.result().numpy())
    #int_fil_text.append(word_reproduce2.result().numpy())
    train_loss_acc.append( train_accloss.result().numpy())
    train_loss_filter.append( train_filloss.result().numpy())
    train_loss_seq.append( train_seqloss.result().numpy())
    train_weighted_loss.append( train_loss.result().numpy())
    train_acc_rate.append( train_accuracy.result().numpy())
    train_ones_num.append( train_ones.result().numpy())
    
    test_loss_acc.append( test_accloss.result().numpy())
    test_loss_filter.append( test_filloss.result().numpy())
    test_loss_seq.append( test_seqloss.result().numpy())
    test_weighted_loss.append( test_loss.result().numpy())
    test_acc_rate.append( test_accuracy.result().numpy())
    test_ones_num.append( test_ones.result().numpy())
    if best_clf<=test_accuracy.result()*100:
        #tf.saved_model.save(model1,saveP1+'_all')
        #model1.save(saveP1,save_format='h5')
        #model2.save(saveP2,save_format='h5')
        #tf.saved_model.save(model2,saveP2+'_all')
        model1.save_weights(saveP1,save_format='tf')
        model2.save_weights(saveP2,save_format='tf')
        best_clf = test_accuracy.result()*100
        print('===MODEL WEIGHTS SAVED===',saveP1,saveP2)
    # Reset the metrics for the next epoch
    #word_reproduce1.reset_states()
    #word_reproduce2.reset_states()
    train_loss.reset_states()
    train_accloss.reset_states()
    train_filloss.reset_states()
    train_seqloss.reset_states()
    train_ones.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accloss.reset_states()
    test_filloss.reset_states()
    test_seqloss.reset_states()
    test_ones.reset_states()
    test_accuracy.reset_states()

Epoch 1, Total Loss: 3.099400758743286, Clf Loss: 3.099398374557495, Filter Loss: 0.5832521915435791, Seq Loss: 0.11047469079494476, Accuracy Rate: 35.04%, Ones Portion: 0.8896543979644775,             Test_Total_Loss: 2.6405112743377686, Test_Clf_Loss: 2.6405086517333984, Test_Filter_Loss: 0.52372145652771, TEST_Seq_Loss: 0.661488950252533, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 0.33921027183532715
===MODEL WEIGHTS SAVED=== ./model/2019112702/model1 ./model/2019112702/model2
Epoch 2, Total Loss: 2.4706435203552246, Clf Loss: 2.4706408977508545, Filter Loss: 0.5493626594543457, Seq Loss: 0.3163779675960541, Accuracy Rate: 37.72%, Ones Portion: 0.6839318871498108,             Test_Total_Loss: 2.4235663414001465, Test_Clf_Loss: 2.4235637187957764, Test_Filter_Loss: 0.5501726269721985, TEST_Seq_Loss: 0.3087596893310547, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 0.6915099024772644
===MODEL WEIGHTS SAVED=== ./model/2019112702/model1 ./model/2019112702/model2
Epoch 3, Total Loss

Epoch 20, Total Loss: 2.270428419113159, Clf Loss: 2.2704250812530518, Filter Loss: 0.5306491255760193, Seq Loss: 0.5876320004463196, Accuracy Rate: 37.51%, Ones Portion: 0.4129648208618164,             Test_Total_Loss: 2.27176570892334, Test_Clf_Loss: 2.271763324737549, Test_Filter_Loss: 0.5299664735794067, TEST_Seq_Loss: 0.6176204681396484, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 0.3829679787158966
Epoch 21, Total Loss: 2.26098895072937, Clf Loss: 2.260986089706421, Filter Loss: 0.5300360321998596, Seq Loss: 0.6052649021148682, Accuracy Rate: 37.74%, Ones Portion: 0.3953196406364441,             Test_Total_Loss: 2.28365421295166, Test_Clf_Loss: 2.283651351928711, Test_Filter_Loss: 0.5360197424888611, TEST_Seq_Loss: 0.5234215259552002, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 0.4770996570587158
Epoch 22, Total Loss: 2.264296531677246, Clf Loss: 2.2642929553985596, Filter Loss: 0.531596839427948, Seq Loss: 0.585905134677887, Accuracy Rate: 37.46%, Ones Portion: 0.414647191

Epoch 40, Total Loss: 2.279104709625244, Clf Loss: 2.279102325439453, Filter Loss: 0.5188926458358765, Seq Loss: 0.7421483397483826, Accuracy Rate: 37.47%, Ones Portion: 0.25852543115615845,             Test_Total_Loss: 2.278980255126953, Test_Clf_Loss: 2.278977870941162, Test_Filter_Loss: 0.5231168866157532, TEST_Seq_Loss: 0.7099225521087646, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 0.29082930088043213
Epoch 41, Total Loss: 2.2751805782318115, Clf Loss: 2.275177240371704, Filter Loss: 0.5184797644615173, Seq Loss: 0.7427811622619629, Accuracy Rate: 37.71%, Ones Portion: 0.25790128111839294,             Test_Total_Loss: 2.2826271057128906, Test_Clf_Loss: 2.2826242446899414, Test_Filter_Loss: 0.5185315608978271, TEST_Seq_Loss: 0.7520356178283691, Test_Accuracy_Rate: 37.80%, Test_Ones_Portion: 0.24864813685417175
Epoch 42, Total Loss: 2.278149127960205, Clf Loss: 2.278146982192993, Filter Loss: 0.5179998874664307, Seq Loss: 0.7454262375831604, Accuracy Rate: 37.52%, Ones Portion: 0

Epoch 60, Total Loss: 2.287653923034668, Clf Loss: 2.287651777267456, Filter Loss: 0.5119679570198059, Seq Loss: 0.7956172227859497, Accuracy Rate: 37.75%, Ones Portion: 0.20512834191322327,             Test_Total_Loss: 2.2871153354644775, Test_Clf_Loss: 2.2871124744415283, Test_Filter_Loss: 0.5114724636077881, TEST_Seq_Loss: 0.807007372379303, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 0.19366595149040222
Epoch 61, Total Loss: 2.2863030433654785, Clf Loss: 2.2863004207611084, Filter Loss: 0.5128798484802246, Seq Loss: 0.7916156053543091, Accuracy Rate: 37.57%, Ones Portion: 0.2090769112110138,             Test_Total_Loss: 2.284849166870117, Test_Clf_Loss: 2.284846305847168, Test_Filter_Loss: 0.5142558217048645, TEST_Seq_Loss: 0.7887012958526611, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 0.21206901967525482
Epoch 62, Total Loss: 2.280674457550049, Clf Loss: 2.2806713581085205, Filter Loss: 0.5137882828712463, Seq Loss: 0.7876949906349182, Accuracy Rate: 37.85%, Ones Portion: 0

Epoch 80, Total Loss: 2.291175603866577, Clf Loss: 2.291172981262207, Filter Loss: 0.4976024925708771, Seq Loss: 0.8488046526908875, Accuracy Rate: 37.69%, Ones Portion: 0.15185202658176422,             Test_Total_Loss: 2.2928431034088135, Test_Clf_Loss: 2.2928409576416016, Test_Filter_Loss: 0.4981556534767151, TEST_Seq_Loss: 0.851800262928009, Test_Accuracy_Rate: 38.96%, Test_Ones_Portion: 0.1489088535308838
===MODEL WEIGHTS SAVED=== ./model/2019112702/model1 ./model/2019112702/model2
Epoch 81, Total Loss: 2.291290044784546, Clf Loss: 2.291287422180176, Filter Loss: 0.5004313588142395, Seq Loss: 0.8429602384567261, Accuracy Rate: 37.97%, Ones Portion: 0.15772387385368347,             Test_Total_Loss: 2.293560743331909, Test_Clf_Loss: 2.293558120727539, Test_Filter_Loss: 0.4964826703071594, TEST_Seq_Loss: 0.8542167544364929, Test_Accuracy_Rate: 39.01%, Test_Ones_Portion: 0.1464722454547882
===MODEL WEIGHTS SAVED=== ./model/2019112702/model1 ./model/2019112702/model2
Epoch 82, Total Los

...

Epoch 1079, Total Loss: 2.3345773220062256, Clf Loss: 2.3345773220062256, Filter Loss: 0.010041311383247375, Seq Loss: 0.9962198138237, Accuracy Rate: 37.78%, Ones Portion: 0.0038939921651035547,             Test_Total_Loss: 2.3586227893829346, Test_Clf_Loss: 2.3586227893829346, Test_Filter_Loss: 0.011789253912866116, TEST_Seq_Loss: 0.9940934777259827, Test_Accuracy_Rate: 37.22%, Test_Ones_Portion: 0.006013978738337755
Epoch 1080, Total Loss: 2.335097551345825, Clf Loss: 2.335097551345825, Filter Loss: 0.010174225084483624, Seq Loss: 0.9960899949073792, Accuracy Rate: 37.78%, Ones Portion: 0.004023573361337185,             Test_Total_Loss: 2.358281135559082, Test_Clf_Loss: 2.358281135559082, Test_Filter_Loss: 0.011784911155700684, TEST_Seq_Loss: 0.9940953254699707, Test_Accuracy_Rate: 37.22%, Test_Ones_Portion: 0.006012406665831804
Epoch 1081, Total Loss: 2.332092046737671, Clf Loss: 2.332092046737671, Filter Loss: 0.01007442083209753, Seq Loss: 0.9961181879043579, Accuracy Rate: 37.77

Epoch 1099, Total Loss: 2.3355183601379395, Clf Loss: 2.3355183601379395, Filter Loss: 0.009964934550225735, Seq Loss: 0.9962552785873413, Accuracy Rate: 37.77%, Ones Portion: 0.00385313224978745,             Test_Total_Loss: 2.3581106662750244, Test_Clf_Loss: 2.3581106662750244, Test_Filter_Loss: 0.011687381193041801, TEST_Seq_Loss: 0.9941492676734924, Test_Accuracy_Rate: 37.22%, Test_Ones_Portion: 0.005965784657746553
Epoch 1100, Total Loss: 2.334296703338623, Clf Loss: 2.334296703338623, Filter Loss: 0.009936338290572166, Seq Loss: 0.9961929321289062, Accuracy Rate: 37.77%, Ones Portion: 0.003920367453247309,             Test_Total_Loss: 2.359208583831787, Test_Clf_Loss: 2.359208583831787, Test_Filter_Loss: 0.011680074036121368, TEST_Seq_Loss: 0.9941515922546387, Test_Accuracy_Rate: 37.22%, Test_Ones_Portion: 0.005963724106550217
Epoch 1101, Total Loss: 2.337137222290039, Clf Loss: 2.337137222290039, Filter Loss: 0.009978820569813251, Seq Loss: 0.996185302734375, Accuracy Rate: 37.7

Epoch 1119, Total Loss: 2.3337242603302, Clf Loss: 2.3337242603302, Filter Loss: 0.009768160991370678, Seq Loss: 0.9963800311088562, Accuracy Rate: 37.77%, Ones Portion: 0.0037338952533900738,             Test_Total_Loss: 2.358189344406128, Test_Clf_Loss: 2.358189344406128, Test_Filter_Loss: 0.01157358754426241, TEST_Seq_Loss: 0.9942164421081543, Test_Accuracy_Rate: 37.22%, Test_Ones_Portion: 0.005906575825065374
Epoch 1120, Total Loss: 2.335392713546753, Clf Loss: 2.335392713546753, Filter Loss: 0.009800074622035027, Seq Loss: 0.996300458908081, Accuracy Rate: 37.77%, Ones Portion: 0.0038074390031397343,             Test_Total_Loss: 2.3600118160247803, Test_Clf_Loss: 2.3600118160247803, Test_Filter_Loss: 0.011569072492420673, TEST_Seq_Loss: 0.9942231774330139, Test_Accuracy_Rate: 37.22%, Test_Ones_Portion: 0.005901684984564781
Epoch 1121, Total Loss: 2.334084987640381, Clf Loss: 2.334084987640381, Filter Loss: 0.009856391698122025, Seq Loss: 0.9962635636329651, Accuracy Rate: 37.77%, 

Epoch 1139, Total Loss: 2.335972785949707, Clf Loss: 2.335972785949707, Filter Loss: 0.00967133417725563, Seq Loss: 0.996351957321167, Accuracy Rate: 37.77%, Ones Portion: 0.0037574267480522394,             Test_Total_Loss: 2.359710931777954, Test_Clf_Loss: 2.359710931777954, Test_Filter_Loss: 0.011456304229795933, TEST_Seq_Loss: 0.9942799806594849, Test_Accuracy_Rate: 37.22%, Test_Ones_Portion: 0.005850082729011774
Epoch 1140, Total Loss: 2.3366289138793945, Clf Loss: 2.3366289138793945, Filter Loss: 0.009856757707893848, Seq Loss: 0.9961929321289062, Accuracy Rate: 37.77%, Ones Portion: 0.003921377472579479,             Test_Total_Loss: 2.3595032691955566, Test_Clf_Loss: 2.3595032691955566, Test_Filter_Loss: 0.011445284821093082, TEST_Seq_Loss: 0.9942823052406311, Test_Accuracy_Rate: 37.22%, Test_Ones_Portion: 0.005848264787346125
Epoch 1141, Total Loss: 2.3328773975372314, Clf Loss: 2.3328773975372314, Filter Loss: 0.009683218784630299, Seq Loss: 0.996364176273346, Accuracy Rate: 37

Epoch 1159, Total Loss: 2.336899518966675, Clf Loss: 2.336899518966675, Filter Loss: 0.009624398313462734, Seq Loss: 0.9963691234588623, Accuracy Rate: 37.77%, Ones Portion: 0.003746556118130684,             Test_Total_Loss: 2.3599796295166016, Test_Clf_Loss: 2.3599796295166016, Test_Filter_Loss: 0.011342810466885567, TEST_Seq_Loss: 0.9943491816520691, Test_Accuracy_Rate: 37.22%, Test_Ones_Portion: 0.005786838475614786
Epoch 1160, Total Loss: 2.3370211124420166, Clf Loss: 2.3370211124420166, Filter Loss: 0.00954380165785551, Seq Loss: 0.9963944554328918, Accuracy Rate: 37.77%, Ones Portion: 0.003723644884303212,             Test_Total_Loss: 2.3596880435943604, Test_Clf_Loss: 2.3596880435943604, Test_Filter_Loss: 0.011335517279803753, TEST_Seq_Loss: 0.9943549036979675, Test_Accuracy_Rate: 37.22%, Test_Ones_Portion: 0.0057812705636024475
Epoch 1161, Total Loss: 2.337895154953003, Clf Loss: 2.337895154953003, Filter Loss: 0.009555195458233356, Seq Loss: 0.9964841604232788, Accuracy Rate: 

### Store process

In [ ]:
save_dir = './results/'+DateID+'/'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir, exist_ok=True)
saveR = save_dir + 'losses_metrics.xlsx'
data = {'train total loss':train_weighted_loss, 'train acc loss':train_loss_acc,
        'train filter loss':train_loss_filter,'train seq loss':train_loss_seq,
        'train acc rate':train_acc_rate, 'train ones num':train_ones_num,
        'test total loss':test_weighted_loss, 'test acc loss':test_loss_acc,
        'test filter loss':test_loss_filter, 'test seq loss': test_loss_seq,
        'test acc rate':test_acc_rate, 'test ones num':test_ones_num
       }
df = pd.DataFrame(data)
df.to_excel(saveR)
print(saveR)
df

## Inference
* Case study

In [ ]:
labelID_dict = {'copper': 6, 'livestock': 28, 'gold': 25, 'money-fx': 19, 'ipi': 30, 'trade': 11, 
                'cocoa': 0, 'iron-steel': 31, 'reserves': 12, 'tin': 26, 'zinc': 37, 'jobs': 34, 
                'ship': 13, 'cotton': 14, 'alum': 23, 'strategic-metal': 27, 'lead': 45, 
                'housing': 7, 'meal-feed': 22, 'gnp': 21, 'sugar': 10, 'rubber': 32, 'dlr': 40,
                'veg-oil': 2, 'interest': 20, 'crude': 16, 'coffee': 9, 'wheat': 5, 'carcass': 15,
                'lei': 35, 'gas': 41, 'nat-gas': 17, 'oilseed': 24, 'orange': 38,
                'heat': 33, 'wpi': 43, 'silver': 42, 'cpi': 18, 'earn': 3, 'bop': 36,
                'money-supply': 8, 'hog': 44, 'acq': 4, 'pet-chem': 39, 'grain': 1, 'retail': 29} # 46個cat
IDlabel_dict = {v: k for k, v in labelID_dict.items()}
word_idx_pd = pd.DataFrame(data=word_index,index=["ID"]).T
word_idx_pd['ori_word'] = word_idx_pd.index 
word_idx_pd = word_idx_pd.set_index(["ID"])
print(word_idx_pd)

def getKeysByValues(listOfValues,dfOfElements=word_idx_pd):
    ori_word_li = []
    for item in listOfValues:
        key = item
#         print(key)
        if key == 0:
            continue
        ori_word = dfOfElements.loc[key][0]
        ori_word_li.append(ori_word)
    return ori_word_li

In [ ]:
word_fil_text = []
word_ori_text = []
from tqdm import tqdm
for text in tqdm(int_ori_text): #text = one epoch
    temp = []
    for t in text: #t = one sample
        temp.append(getKeysByValues(list(t)))
    word_ori_text.append(temp)
for text in tqdm(int_fil_text):
    temp = []
    for t in text:
        temp.append(getKeysByValues(list(t)))
    word_fil_text.append(temp)

save_path = save_dir + "original_words.pkl"
pickle.dump(file=open(save_path,'wb'),obj=word_ori_text)
save_path = save_dir + "filterUsed_words.pkl"
pickle.dump(file=open(save_path,'wb'),obj=word_fil_text)
print(save_path)

In [ ]:
col = ['epoch#','text_id','ori_text','filter_text','label']
df = pd.DataFrame(columns=col)
for epoch,(text_ori,text_fil,labelID) in tqdm(enumerate(zip(word_ori_text,word_fil_text,int_label))):
    for idx,(t_o,t_f,label) in enumerate(zip(text_ori,text_fil,labelID)):
        temp = pd.Series([epoch,idx," ".join(t_o)," ".join(t_f),IDlabel_dict[label]],col)
        df = df.append(temp,ignore_index=True)

save_path = save_dir + "caseStudy_result.xlsx"
df.to_excel(save_path,index=False)
print(save_path)
df

4

# Deprecated

In [161]:
save_path = save_dir + "caseStudy_result.xlsx"
df.to_excel(save_path,index=False)
save_path = save_dir + "original_words.pkl"
pickle.dump(file=open(save_path,'wb'),obj=word_ori_text)
save_path = save_dir + "filterUsed_words.pkl"
pickle.dump(file=open(save_path,'wb'),obj=word_fil_text)

In [ ]:
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #phase1: pred_imp ; phase2:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)
for test_text, test_labels in valid_ds:
    test_step(test_text, test_labels)

In [7]:
for test_text, test_labels in valid_ds:
    print(test_text)
    print(test_labels)
    break
#     test_step(test_text, test_labels)

NameError: name 'valid_ds' is not defined

In [11]:
convert_li = getKeysByValues(testX[0])
convert_li

['the',
 'in',
 'compared',
 'analysts',
 'etl',
 'at',
 'may',
 'said',
 'si',
 'offshore',
 'a',
 'but',
 'a',
 'ohio',
 'and',
 'publicly',
 'its',
 'sent',
 '380',
 'plan',
 'bank',
 'mln',
 '1',
 '756',
 'not',
 'note',
 'was',
 'finding',
 'eased',
 '80',
 'in',
 'compared',
 'analysts',
 'as',
 'main',
 'and',
 'origins',
 'be',
 'in',
 'borrowers',
 'said',
 'in',
 'hearings',
 'said',
 'becoming',
 '3',
 'as',
 'of',
 'was',
 'in',
 'worsen',
 'analysts',
 'that',
 'of',
 'offshore',
 'flight',
 'for',
 'during',
 'time',
 'official',
 'mln',
 'a',
 'for',
 'capital',
 'in',
 'par',
 'said',
 'in',
 'worsen',
 'sought',
 'bank',
 'sales',
 'commodity',
 'common',
 'above',
 'a',
 'merchants',
 'amounted',
 'already',
 'in',
 'capital',
 'mln',
 'in',
 'compared',
 'analysts',
 '3',
 'in',
 'worsen',
 '87',
 '41',
 'for',
 'hearings',
 'said',
 'becoming',
 'importers',
 'offshore',
 'flight',
 'be',
 'in',
 'has',
 'would',
 '24',
 '32',
 'firms',
 'of',
 'a',
 'only',
 'cattl

In [12]:
reuters.get_word_index()

{'mdbl': 10996,
 'fawc': 16260,
 'degussa': 12089,
 'woods': 8803,
 'hanging': 13796,
 'localized': 20672,
 'sation': 20673,
 'chanthaburi': 20675,
 'refunding': 10997,
 'hermann': 8804,
 'passsengers': 20676,
 'stipulate': 20677,
 'heublein': 8352,
 'screaming': 20713,
 'tcby': 16261,
 'four': 185,
 'grains': 1642,
 'broiler': 20680,
 'wooden': 12090,
 'wednesday': 1220,
 'highveld': 13797,
 'duffour': 7593,
 '0053': 20681,
 'elections': 3914,
 '270': 2563,
 '271': 3551,
 '272': 5113,
 '273': 3552,
 '274': 3400,
 'rudman': 7975,
 '276': 3401,
 '277': 3478,
 '278': 3632,
 '279': 4309,
 'dormancy': 9381,
 'errors': 7247,
 'deferred': 3086,
 'sptnd': 20683,
 'cooking': 8805,
 'stratabit': 20684,
 'designing': 16262,
 'metalurgicos': 20685,
 'databank': 13798,
 '300er': 20686,
 'shocks': 20687,
 'nawg': 7972,
 'tnta': 20688,
 'perforations': 20689,
 'affiliates': 2891,
 '27p': 20690,
 'ching': 16263,
 'china': 595,
 'wagyu': 16264,
 'affiliated': 3189,
 'chino': 16265,
 'chinh': 16266,
 '

,ori_word
ID,
10996,mdbl
16260,fawc
12089,degussa
8803,woods
13796,hanging
...,...
16258,rotting
10995,pods
2849,emery


In [1047]:
test_text[0][0].numpy()

1

In [1049]:
word_idx_pd.loc[test_text[0][0].numpy()][0]

'the'

In [1036]:
word_idx_pd

,ID
mdbl,10996
fawc,16260
degussa,12089
woods,8803
hanging,13796
...,...
rotting,16258
pods,10995
emery,2849
northerly,30979


In [714]:
np.array(train_weighted_loss[0])

array(None, dtype=object)

In [732]:
kk = np.array(test_accloss.result())
kk

array(2.3583207, dtype=float32)

In [735]:
test_accloss.result().numpy()

2.3583207

In [523]:
# model1.save('./model/emb_layer') #tf.keras.models.load_model
tf.saved_model.save(model1,'./model/emb_layer')
model3 = tf.keras.models.load_model('./model/emb_layer')

TypeError: unsupported callable

In [524]:
model3 = model1.load_weights('./model/emb_layer_weight')

AttributeError: 'CheckpointLoadStatus' object has no attribute 'summary'

In [527]:
model1.emb.set_weights(model3.emb.get_weights())

In [526]:
model3 = model1
model3.load_weights('./model/emb_layer_weight')
model3.summary()

Model: "base_model_1_58"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_58 (Masking)         multiple                  0         
_________________________________________________________________
glove_emb (Embedding)        multiple                  2138112   
_________________________________________________________________


ValueError: You tried to call `count_params` on common_extract, but the layer isn't built. You can build it manually via: `common_extract.build(batch_input_shape)`.

* 在某些參數的情況下0 1是train不起來的但是weight可以
    * alpha = 0.0, beta=0.6
    * init_w = tensorflow.keras.initializers.Constant(value=0.9), init_b = tensorflow.keras.initializers.Constant(value=0.7)
* 0/1 with emb比較容易overfit。如果是weight的比較沒那麼嚴重

* 不用sigmoid或是hard_sigmoid。改良relu + linear，並拆成兩個model，把round前面多加上clip
    * sigmoid中間的變化太快(一瞬間就會掉到0或是1)，改成relu在>0~無限大(linear為了還在0~1)再去clip再round，可以看到每個epoch的變化
* 1st phase的beta一定要>=0.6否則不會動，ones#都會是0
    * beta=1 (放0 1進去): weight init設成1也沒用，但把bias設成1就會一開始都是ones#=1了。weight=1 bias=0.2都匯市0，bias=0.3會是0.87。0.6/0.3都是0。0.8/0.3差不多是0.5(但是train很慢acc進步很慢就是了)。0.5/0.5是從0.01開始往上升 (0 1放進去會比較難train是因為它的變化量太大，一下就是有或沒有，所以clf可能學不好，但如果是weight每次gradient進步的都是一小點就會比較容易上升)
    * beta=0.6 (放0 1進去): 0.8/0.5都是0。0.9/0.8 從0.9一直到0。0.9/0.6差不多是從0.5但又有時候到0.7都是0.0(很難train，Nadam換個opt有時候沒用。EX變成adam 0.9/0.8才有0.96開始但如果0.9/0.75變成0開始。Rmsprop 0.9/0.8又是從0.95開始往下)。但如果都改成傳入weight就都沒問題。ones一開始大概0.5 weight平均，也不會卡住
* 建議: 先訓練embedding weight matrix，但是要看goal是要怎樣的matrix
* 其實他不管幾%都會train得很好，除非固定embedding，或是用更弱的clf
* 若設定alpha，就像是regularizer term (penalty)，設越大drop越多
* 一開始先很多ones，再越來越少個

* 同一個opt若加入transform就會train不起來
* 兩個不同的opt加入transform也會train不起來 (persistent、non-persis都不行)，且與BN無關